# Climatological tracking of IVT and AR objects using the MOAAP tracking algorithm
- Load IVT tracking data in chunks of 7 months with 1 month overlap between chunks
- Track individual data chunks and correct the pickle and netCdf4 files afterwards, so that each final corrected file corresponds to 6 month, from 1.1-1.7 and 1.7-1-1 respectively
- Tracking should be done on a rotated grid to avoid strange behaviour around the pole when e.g. using a regular grid
- Tracking data are remapped to 33km resolution


In [5]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import datetime
import cartopy
import cartopy.crs as ccrs
import pickle
#import Tracking_Functions
from dateutil import relativedelta
import os

import src.Tracking_Functions as Tracking_Functions
from src.TrackingDataLoader import * 
from src.utils import * 
from src.Corrections import * 
from src.Enumerations import Experiments

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Settings

In [6]:
#data_path = '/work/aa0049/a271109/spice-v2.1/chain/work/polarres_wp3_cmip_CNRM/post/yearly/'
#exp = Data.ICON_NORESM_EXP.value


suffix = 'remapped_3x'
file_type = '.nc'

#######ICON########

exp = Experiments.ICON_NORESM_CONTROL.value

first_year = 1984
last_year = 1985

data_path = exp.path
output_path = exp.IVTobj_out_path
output_path = "/work/aa0238/a271093/scratch/test/"

output_file_name_temp = f'MOAPP_ICON_100and85controlperc_{suffix}'

#output_path = '/work/aa0238/a271093/scratch/Track_test/'
threshold_file = exp.IVT_thresh_path+exp.IVT_thresh_file_85

ds_ivt_pctl=xr.open_dataset(threshold_file,decode_times=False)
IVTtrheshold=ds_ivt_pctl.IVT_85perc.values



#####ERA5######

#first_year = 1979
#last_year = 2022


#data_path = '/work/aa0238/a271093/data/ERA5/1979-2023/ICON_remapped_3x/'
#output_path = '/work/aa0238/a271093/results/MOAAP/IVT_Tracking/ERA5_ICON_remapped_3x/'
#output_file_name_temp = f'MOAPP_ERA5_100and85controlperc_{suffix}'
#threshold_file = threshold_path+'ivt_percentile_mlauer_removed-ens-lev_remapbilWP3domain_3dx3dy.nc'

#ds_ivt_pctl=xr.open_dataset(threshold_file,decode_times=False)
#IVTtrheshold=ds_ivt_pctl.ivt1.values

#######ICON SSP###########
exp = Experiments.ICON_NORESM_SSP.value

first_year = exp.year_start
last_year = exp.year_end

data_path = exp.path_IVT
output_path = exp.IVTobj_out_path
output_path = "fff"

output_file_name_temp = f'MOAPP_ICON_100and85NorESMsspperc_{suffix}'

threshold_file = exp.IVT_thresh_path+exp.IVT_thresh_file_85

ds_ivt_pctl=xr.open_dataset(threshold_file,decode_times=False)
IVTtrheshold=ds_ivt_pctl.IVT_85_perc.values


###### ICON-ERA5###########
#exp = Experiments.ICON_ERA5.value

#first_year = exp.year_start
#last_year = exp.year_end

#data_path = exp.path_IVT
#output_path = exp.IVTobj_out_path
#output_path = "/work/aa0238/a271093/scratch/test/"

#output_file_name_temp = f'MOAPP_ICON_ERA5_100and85ERA5perc_{suffix}'

#threshold_file = exp.IVT_thresh_path+exp.IVT_thresh_file_85

#ds_ivt_pctl=xr.open_dataset(threshold_file,decode_times=False)
#IVTtrheshold=ds_ivt_pctl.ivt1.values

In [7]:
start_date_list, end_date_list = create_datetime_lists(first_year,last_year) 
first_processed_date = start_date_list[0]
last_processed_date = end_date_list[-1]

In [8]:
output_path

'fff'

In [9]:

IVTuLoader= TrackingDataLoader('IVTu', data_path, suffix=suffix, file_type = file_type)
IVTvLoader= TrackingDataLoader('IVTv', data_path, suffix=suffix, file_type = file_type)


dict_keys_offset = 0

for start_date, end_date in zip(start_date_list, end_date_list):
    
    print ("\n \n \n \n")
    
    IVTuLoader.start_date = start_date
    IVTvLoader.start_date = start_date
    
    IVTuLoader.end_date = end_date
    IVTvLoader.end_date = end_date

    print (IVTuLoader.start_date, IVTuLoader.end_date)
    

    IVTudata=IVTuLoader.load_datasets(rm_nc=True)
    IVTvdata=IVTvLoader.load_datasets(rm_nc=True)


    rLon = xr.broadcast(IVTudata.rlon, IVTudata.rlat)[0].values.T
    rLat = xr.broadcast(IVTudata.rlon, IVTudata.rlat)[1].values.T

    Lon = xr.broadcast(IVTudata.lon, IVTudata.lat)[0].values
    Lat = xr.broadcast(IVTudata.lon, IVTudata.lat)[1].values

    Mask=1*(rLat>-999)
    Time_sel = IVTuLoader.time_sel
    
    output_file_name = f'{output_file_name_temp}_{get_datetime_str(start_date)}-{get_datetime_str(end_date)}'

    

    Tracking_Functions.moaap(Lon = rLon,                            # 2Dlongitude grid centers
                              Lat = rLat,                           # 2D latitude grid spacing
                              Time = Time_sel,                      # datetime vector of data
                              dT = 1,                               # integer - temporal frequency of data [hour]
                              Mask = Mask,                          # mask with dimensions [lat,lon] defining analysis region

                              ivte = IVTudata.IVTu.values,          # zonal integrated vapor transport [kg m-1 s-1]
                              ivtn = IVTvdata.IVTv.values,          # meidional integrated vapor transport [kg m-1 s-1]
                              regular_Lon = Lon,
                              regular_Lat = Lat,
                              IVTtrheshold = IVTtrheshold,          # Integrated water vapor transport threshold for AR detection [kg m-1 s-1]
                                                                    # JLa: additionall fixed threshold 100 in code

                              DataName = output_file_name,
                              OutputFolder=output_path ,
                              dict_keys_offset = dict_keys_offset
                            )
                       
                             
    cleanup_dicts(output_path,
                  output_file_name_temp,
                  start_date,
                  end_date, 
                  last_processed_date,
                  type_='IVT'
                 )
        
    cleanup_dicts(output_path,
                  output_file_name_temp,
                  start_date,
                  end_date, 
                  last_processed_date,
                  type_='ARs'
                 )
    
    correct_nc_file(output_path,
                    output_file_name_temp, 
                    start_date,
                    end_date,
                    last_processed_date
                   )
                             
    dict_keys_offset +=5000


 
 
 

2014-01-01 00:00:00 2014-08-01 00:00:00
 
The provided variables allow tracking the following phenomena
 
|  phenomenon  | tracking |
---------------------------
   Jetstream   |   no
   PSL CY/ACY  |   no
   Z500 CY/ACY |   no
   COLs        |   no
   IVT ARs     |   yes
   MS ARs      |   no
   Fronts      |   no
   TCs         |   no
   MCSs        |   no
   Equ. Waves  |   no
---------------------------
 
======> track IVT streams and atmospheric rivers (ARs)
        3505 object found
        break up long living IVT objects that have many elements


  1%|          | 6/639 [00:00<00:21, 29.09it/s]


KeyboardInterrupt: 

In [3]:
import xarray as xr
import glob
files = glob.glob("/work/aa0238/a271093/data/Jan_runs/ICON_NorESM_ssp/NorESM_ssp_remapped_3x/IVTu/*.nc")

full_var_ds = xr.open_mfdataset(files)

In [10]:
Time_sel

array([datetime.datetime(2014, 1, 1, 1, 0),
       datetime.datetime(2014, 1, 1, 2, 0),
       datetime.datetime(2014, 1, 1, 3, 0), ...,
       datetime.datetime(2014, 7, 31, 21, 0),
       datetime.datetime(2014, 7, 31, 22, 0),
       datetime.datetime(2014, 7, 31, 23, 0)], dtype=object)

In [ ]:
                    var_ds = full_var_ds.sel(
                        time=slice(start_date, end_date + relativedelta(months=1))
                    ).load()